In [2]:
# Imports and paths
import pandas as pd
import numpy as np
import altair as alt

from IPython.display import display

# Same directory structure as the shared Google Drive folder
DATA_ROOT = '../data/'
BEER_ADVOCATE_PATH = f'{DATA_ROOT}BeerAdvocate/'
BEER_ADVOCATE_CSV = f'{BEER_ADVOCATE_PATH}beer_reviews.csv'

# So that long texts are visible when displaying DataFrames
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv(BEER_ADVOCATE_CSV)

# Proper datetimes are more readable than unix timestamps
df['review_time'] = pd.to_datetime(df['review_time'], unit='s')

df.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,2009-02-16 20:57:03,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,2009-03-01 13:44:57,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,2009-03-01 14:10:04,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,2009-02-15 19:12:25,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,2010-12-30 18:53:26,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1586614 entries, 0 to 1586613
Data columns (total 13 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   brewery_id          1586614 non-null  int64         
 1   brewery_name        1586599 non-null  object        
 2   review_time         1586614 non-null  datetime64[ns]
 3   review_overall      1586614 non-null  float64       
 4   review_aroma        1586614 non-null  float64       
 5   review_appearance   1586614 non-null  float64       
 6   review_profilename  1586266 non-null  object        
 7   beer_style          1586614 non-null  object        
 8   review_palate       1586614 non-null  float64       
 9   review_taste        1586614 non-null  float64       
 10  beer_name           1586614 non-null  object        
 11  beer_abv            1518829 non-null  float64       
 12  beer_beerid         1586614 non-null  int64         
dtypes: datetime6

# Exploramos un poco los datos

In [5]:
rate_of_reviews_by_brewery = (
  (
    100 * df.groupby('brewery_name').size() / len(df)
  )
  .reset_index()
  .rename(columns={0: 'rate'})
  .sort_values('rate', ascending=False)
)
rate_of_reviews_by_brewery

print(f"Hay {len(rate_of_reviews_by_brewery)} cervecerias distintas.")

alt.Chart(rate_of_reviews_by_brewery.iloc[:50]).mark_line().encode(
    x=alt.X('brewery_name', sort='-y'),
    y='rate',    
).properties(title='Relación de cantidad de reviews por cervecería sobre el total').interactive()

Hay 5742 cervecerias distintas.


alt.Chart(...)

In [6]:
categorical_col = 'beer_style'
rate_of_reviews_by_style = (
  (
    100 * df.groupby(categorical_col).size() / len(df)
  )
  .reset_index()
  .rename(columns={0: 'rate'})
  .sort_values('rate', ascending=False)
)
rate_of_reviews_by_style

print(f"Hay {len(rate_of_reviews_by_style)} estilos de cerveza distintos.")

alt.Chart(rate_of_reviews_by_style.iloc[:50]).mark_line(color='green').encode(
    x=alt.X(categorical_col, sort='-y'),
    y='rate',    
).properties(
    title='Relación de cantidad de reviews por estilo de cerveza',    
).interactive()

Hay 104 estilos de cerveza distintos.


alt.Chart(...)

In [62]:
len(df['beer_name'].unique())

56857

In [65]:
len(df['beer_beerid'].unique())

66055

In [24]:
col_group_by, col_unique = 'beer_name', 'beer_beerid'
col_new_unique_count = 'breweries_offering'
df_beers_by_name = (
    df.groupby([col_group_by])
    .apply(lambda dfg: len(dfg[col_unique].unique()))
    .reset_index()
    .rename(columns={0: col_new_unique_count})
)
df_beers_by_name

alt.Chart(
    df_beers_by_name.sort_values(col_new_unique_count, ascending=False).iloc[:100]
).mark_line(color='orange').encode(
    x=alt.X(col_group_by, sort='-y'),
    y=col_new_unique_count,    
).properties(
    title='Cantidad de cervecerías que ofrecen una cerveza',    
).interactive()

alt.Chart(...)

## Separamos las entidades en distintos DataFrames por conveniencia

In [8]:
df_beer = df[['beer_beerid', 'brewery_id', 'beer_name', 'beer_style', 'beer_abv']].drop_duplicates(subset='beer_beerid').set_index('beer_beerid')
df_brewery = df[['brewery_id', 'brewery_name']].drop_duplicates(subset='brewery_id').set_index('brewery_id')
df_review = df[list(set(['beer_beerid', 'brewery_id'] + list(set(df.columns) - set(df_beer.columns) - set(df_brewery.columns))))]

display(df_beer)
display(df_brewery)
display(df_review)

,brewery_id,beer_name,beer_style,beer_abv
beer_beerid,,,,
47986,10325,Sausa Weizen,Hefeweizen,5.0
48213,10325,Red Moon,English Strong Ale,6.2
48215,10325,Black Horse Black Beer,Foreign / Export Stout,6.5
47969,10325,Sausa Pils,German Pilsener,5.0
64883,1075,Cauldron DIPA,American Double / Imperial IPA,7.7
...,...,...,...,...
38160,14359,Highland Porter,American Porter,6.0
71234,14359,Baron Von Weizen,Hefeweizen,5.0
48360,14359,Resolution #2,Belgian Strong Pale Ale,NaN


,brewery_name
brewery_id,
10325,Vecchio Birraio
1075,Caldera Brewing Company
163,Amstel Brouwerij B. V.
1454,Broad Ripple Brew Pub
850,Moon River Brewing Company
...,...
21727,Gattopardo Cervejaria
6785,Brauerei Lasser GmbH
12566,Wissey Valley Brewery


,review_overall,beer_beerid,review_taste,review_profilename,brewery_id,review_palate,review_aroma,review_appearance,review_time
0,1.5,47986,1.5,stcules,10325,1.5,2.0,2.5,2009-02-16 20:57:03
1,3.0,48213,3.0,stcules,10325,3.0,2.5,3.0,2009-03-01 13:44:57
2,3.0,48215,3.0,stcules,10325,3.0,2.5,3.0,2009-03-01 14:10:04
3,3.0,47969,3.0,stcules,10325,2.5,3.0,3.5,2009-02-15 19:12:25
4,4.0,64883,4.5,johnmichaelsen,1075,4.0,4.5,4.0,2010-12-30 18:53:26
...,...,...,...,...,...,...,...,...,...
1586609,5.0,33061,4.0,maddogruss,14359,4.0,4.0,3.5,2006-11-05 00:01:32
1586610,4.0,33061,4.0,yelterdow,14359,2.0,5.0,2.5,2006-10-17 01:29:26
1586611,4.5,33061,4.0,TongoRad,14359,3.5,3.5,3.0,2006-10-13 01:21:53
1586612,4.0,33061,4.5,dherling,14359,4.5,4.5,4.5,2006-10-05 04:37:24


In [16]:
df_beer_review_count = df_beer.merge(df_brewery, on='brewery_id')
df_beer_review_count['review_count'] = df_review.groupby('beer_beerid').size()
df_beer_review_count['full_name'] = df_beer_review_count['brewery_name'] + ' || ' + df_beer_review_count['beer_name']

print(f"Cervezas con una sola review: {len(df_beer_review_count[df_beer_review_count['review_count'] == 1])}")

alt.Chart(
    df_beer_review_count.sort_values('review_count', ascending=False).iloc[:100]
).mark_line(color='red').encode(
    x=alt.X(
        'full_name', 
        sort='-y', 
        # Esto es para que los nombres largos no se corten en el eje X.
        axis=alt.Axis(labelLimit=550)
    ),
    y='review_count',    
).properties(
    title='Cantidad de reviews por cerveza',    
).interactive()

Cervezas con una sola review: 18192


alt.Chart(...)

In [9]:
def keep_groups(dfg):
    return len(dfg) >= 100

df_rating_overall_stats = (
    df_review
    .groupby(['beer_beerid'])
    .filter(keep_groups)
    .groupby(['beer_beerid'])
    .agg(
        avg_review_overall=pd.NamedAgg(column='review_overall', aggfunc='mean'),
        median_review_overall=pd.NamedAgg(column='review_overall', aggfunc='median'),
        count=pd.NamedAgg(column='review_overall', aggfunc=len)
    )
).sort_values('avg_review_overall', ascending=False).merge(df_beer['beer_name'], left_index=True, right_index=True)
df_rating_overall_stats

,avg_review_overall,median_review_overall,count,beer_name
beer_beerid,,,,
56082,4.630952,4.5,252.0,Citra DIPA
36316,4.628205,4.5,156.0,Cantillon Blåbær Lambik
16814,4.625800,4.5,469.0,Heady Topper
41928,4.620536,4.5,112.0,Deviation - Bottleworks 9th Anniversary
1545,4.617925,4.5,1272.0,Trappist Westvleteren 12
...,...,...,...,...
37389,1.931159,1.0,138.0,Budweiser Chelada
26049,1.924274,1.5,482.0,Wild Blue (Blueberry Lager)
37424,1.710526,1.0,114.0,Bud Light Chelada


In [10]:
display(df_rating_overall_stats.iloc[:3])
display(df_rating_overall_stats.iloc[-3:])

,avg_review_overall,median_review_overall,count,beer_name
beer_beerid,,,,
56082,4.630952,4.5,252.0,Citra DIPA
36316,4.628205,4.5,156.0,Cantillon Blåbær Lambik
16814,4.625800,4.5,469.0,Heady Topper


,avg_review_overall,median_review_overall,count,beer_name
beer_beerid,,,,
37424,1.710526,1.0,114.0,Bud Light Chelada
38763,1.681818,1.5,132.0,Original C Cave Creek Chili Beer - Cerveza Con Chili
2213,1.488764,1.0,267.0,Crazy Ed's Cave Creek Chili Beer


In [15]:
df_rating_avb = df_rating_overall_stats.merge(df_beer, left_index=True, right_index=True)
alt.Chart(df_rating_avb).mark_circle().encode(
    alt.X('avg_review_overall', scale=alt.Scale(zero=False)),
    alt.Y('beer_abv', scale=alt.Scale(zero=False, padding=1)),
    color='beer_style',
    size='count',
    tooltip='beer_name_x'
).properties(height=600, width=1200).interactive()

alt.Chart(...)

## Resumen
### Cuantas
- ..cervezas distintas hay?
  - 66,055
- ..cervecerias distintas hay?
  - 5,840

### Cuales
- ..son más ofrecidas por las cervecerias?
  - **Oktoberfest**, **Pale Ale**, **Oatmeal South**
- ..tienen más reviews?
  1. Coastal Extreme Brewing Co. || **Castle Hill Windward Weiss**
  2. Harboes Bryggeri A/S || **Harboe Bjørnebryg (Bear Beer)**
  3. Rinkuškiai Alaus Darykloje || **Lobster Lovers Beer**
  
- ..cervezas tienen mejor/peor puntaje?
  - Mejores
    - **Citra DIPA**
    - **Cantillon Blåbær Lambik**
    - **Heady Topper**
  - Peores
    - **Crazy Ed's Cave Creek Chili Beer**
    - **Original C Cave Creek Chili Beer - Cerveza Con Chili**
    - **Bud Light Chelada**     
- Nivel de alcohol vs rating promedio:
  - Las cervezas de alta graduación alcohólica empiezan a aparecer más con ratings promedio
    mayores a 3.5
  - Los ratings promedio menores a 3.5 están dominados por las cervezas de graduación media-baja (6% para abajo)
